# Portfolio Project: Safety Option

**Names:** Deniz Makul, Richard Sha, Mitchel Shen

**Due:** 11/25/2022 at 11:59pm ET on Dropbox

**What to submit:** your .ipynb (python code in jupyter notebooks) file as well as a pdf with all the output from your code.  




In [64]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from threading import Thread
import time

In [65]:
#Getting tickers from csv file
csv_tickers = pd.read_csv('Tickers_Example.csv', header = None)

In [66]:
# Returns a list of of months that have less than 20 trading days

volume_start_date = '2022-01-01'
volume_end_date = '2022-10-31'

def remove_month(ticker):
    hist = ticker.history(start = volume_start_date, end = volume_end_date)
    
    months_to_remove = []
    for i in range(1,11,1):
        if i <= 9: #appends a 0 if single digit
            i = str(0) + str(i)
        df = hist.filter(like='2022-' + str(i), axis=0)
        if len(df.index) < 20:
            months_to_remove.append('2022-' + str(i))
    return months_to_remove

In [67]:
volume_index_drop = []

def filter_stocks(ticker, i):
    stock_data = yf.Ticker(ticker)
    stock_hist = stock_data.history(start = volume_start_date, end = volume_end_date, interval = '1mo', Threads = True)
    
    if stock_hist.empty:
        volume_index_drop.append(i)
        return
    
    stock_volume = stock_hist['Volume']
    stock_volume = stock_volume.dropna()
    stock_volume = stock_volume.drop(remove_month(stock_data))
    
    mean = (stock_volume.mean())

    if mean<200000:
        volume_index_drop.append(i)         
    elif(stock_data.info['market'] != 'us_market'):
        volume_index_drop.append(i)
    
    #volume_index_drop

In [68]:
#Drops the tickers that are delisted or have a average monthly volume from 
# January 1st to October 31st of less that 200,000

# Sending in the first ticker to determine which month to remove

thread_list = []

for i in range(len(csv_tickers)):
    ticker = csv_tickers[0][i]
    filter_thread = Thread(target = filter_stocks, args = (ticker,i,))
    thread_list.append(filter_thread)
    filter_thread.start()
    
for i in thread_list:
    i.join()

print(volume_index_drop)

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
[4, 16, 40, 47, 56, 48, 54]


In [69]:
#Drops invalid tickers
csv_tickers.drop(index = volume_index_drop, inplace = True)
csv_tickers.reset_index(drop = True, inplace = True)
csv_tickers.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AIG


In [70]:
close_all = pd.DataFrame()

def stock_history(ticker):
    print(ticker)
    close_start = '2022-01-01'
    close_end = '2022-10-31'
    stock_data = yf.Ticker(ticker)
    stock_hist = stock_data.history(start = close_start, end = close_end, interval = '1d')
    stock_close = stock_hist['Close']
    stock_close = stock_close.dropna()
    close_all[ticker] = stock_close

In [71]:
# Mitchel

thread_list = []

for i in range(0,len(csv_tickers)):
    ticker = csv_tickers[0][i]
    stock_history(ticker)
    
close_all.head()

AAPL
ABBV
ABT
ACN
AIG
AMZN
AXP
BA
BAC
BIIB
BK
BLK
BMY
C
CAT
CL
CMCSA
COF
COP
COST
CSCO
CVS
GM
GOOG
JPM
KMI
KO
LLY
LMT
MO
MON
MRK
MS
MSFT
NEE
NKE
ORCL
OXY
PEP
PFE
PG
PM
PYPL
QCOM
SBUX
SLB
SO
SPG
T
TGT
TXN
UNH
UNP
UPS
USB


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,180.959732,130.341782,136.745361,401.729431,56.390324,170.404495,166.056244,207.860001,45.393047,244.139999,...,31.361927,65.578423,153.253540,17.874916,227.143707,185.358856,497.457550,243.742813,206.660706,55.597622
2022-01-04,178.663071,130.091568,133.529343,398.858582,57.972282,167.522003,171.387100,213.630005,47.172203,241.729996,...,32.884541,65.578423,155.815720,18.022526,229.317703,185.553360,486.186798,247.933548,210.329269,57.053818
2022-01-05,173.910660,130.774918,132.929398,391.834412,57.176388,164.356995,169.560776,213.070007,46.376007,239.270004,...,32.884541,65.655380,153.951416,18.423185,224.117752,181.682800,484.988434,248.681183,207.686722,56.752869
2022-01-06,171.007492,130.158905,132.909729,372.912567,58.158970,163.253998,171.114563,211.339996,47.309818,237.300003,...,33.665623,65.578423,155.987808,18.352894,226.556152,181.624435,465.140808,249.369827,209.700104,58.296444
2022-01-07,171.176529,129.822067,133.322800,365.760132,59.681973,162.554001,172.579300,215.500000,48.341927,232.600006,...,34.634560,66.030556,154.180878,18.979885,225.997971,174.505737,454.196899,250.638855,211.171402,59.218700


In [72]:
close_all_pct = close_all.pct_change() * 100
close_all_pct.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,-1.269156,-0.191967,-2.351830,-0.714623,2.805373,-1.691559,3.210272,2.775909,3.919445,-0.987140,...,4.854974,0.000000,1.671857,0.825792,0.957102,0.104934,-2.265671,1.719327,1.775163,2.619169
2022-01-05,-2.659985,0.525284,-0.449298,-1.761068,-1.372888,-1.889309,-1.065614,-0.262134,-1.687850,-1.017661,...,0.000000,0.117352,-1.196480,2.223104,-2.267575,-2.085955,-0.246482,0.301546,-1.256386,-0.527483
2022-01-06,-1.669344,-0.471048,-0.014796,-4.829041,1.718510,-0.671098,0.916360,-0.811945,2.013565,-0.823338,...,2.375226,-0.117215,1.322750,-0.381538,1.088000,-0.032125,-4.092392,0.276919,0.969432,2.719819
2022-01-07,0.098848,-0.258790,0.310790,-1.917993,2.618689,-0.428778,0.855998,1.968394,2.181594,-1.980614,...,2.878120,0.689454,-1.158379,3.416307,-0.246377,-3.919461,-2.352816,0.508894,0.701620,1.582012


In [73]:
#Expected Return
expected_return = close_all_pct.mean()
expected_return.head()

AAPL   -0.049038
ABBV    0.071278
ABT    -0.138700
ACN    -0.140936
AIG     0.027678
dtype: float64

In [74]:
expected_return_from_0 = abs(0 - expected_return)
expected_return_from_0

expected_return_sorted = sorted(expected_return_from_0.items(), key=lambda x:x[1])

In [75]:
std_lst = close_all_pct.std()
std_lst = sorted(std_lst.items(), key=lambda x:x[1])

In [76]:
correlation_pct_stocks = pd.DataFrame()
index = []

for i in range(len(close_all_pct.columns)):
    
    correlation_temp = {}
    
    for j in range(len(close_all_pct.columns)):
        correlation_df = pd.DataFrame({
            csv_tickers[0][i]: close_all_pct[csv_tickers[0][i]]
        })
        
        correlation_df[csv_tickers[0][j]] = close_all_pct[csv_tickers[0][j]]
        correlation = correlation_df.corr()[csv_tickers[0][i]][csv_tickers[0][j]]

        correlation_temp[csv_tickers[0][j]] = correlation
        
    correlation_pct_stocks[csv_tickers[0][i]] = list(correlation_temp.values())
    index.append(csv_tickers[0][i])


correlation_pct_stocks.set_index([pd.Index(index)])
correlation_pct_stocks.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
0,1.000000,0.206871,0.610150,0.788787,0.566180,0.662325,0.640718,0.576470,0.568125,0.298192,...,0.209095,0.376867,0.600640,0.384481,0.506570,0.730380,0.518132,0.477781,0.540758,0.468527
1,0.206871,1.000000,0.454250,0.312444,0.303832,0.199121,0.262372,0.110946,0.297603,0.231642,...,0.172512,0.381071,0.256924,0.268500,0.262418,0.235727,0.533026,0.247252,0.207049,0.378954
2,0.610150,0.454250,1.000000,0.683649,0.531133,0.486219,0.582679,0.430351,0.568027,0.331139,...,0.118772,0.446448,0.520299,0.373109,0.423279,0.528107,0.591592,0.493834,0.472782,0.549303
3,0.788787,0.312444,0.683649,1.000000,0.602921,0.679004,0.650519,0.548784,0.591080,0.385234,...,0.203884,0.413858,0.645052,0.364540,0.548192,0.737162,0.581465,0.560528,0.578152,0.526037
4,0.566180,0.303832,0.531133,0.602921,1.000000,0.459222,0.739157,0.540230,0.761078,0.243192,...,0.390822,0.289413,0.608583,0.373590,0.342745,0.493510,0.401822,0.515643,0.511382,0.735766


In [77]:
correlation_pct_mean = pd.DataFrame()
correlation_pct_mean = correlation_pct_stocks.sum()
correlation_pct_mean = correlation_pct_mean - 1
correlation_pct_mean = correlation_pct_mean/(len(correlation_pct_stocks) - 1)

correlation_pct_mean.head()

AAPL    0.498874
ABBV    0.290250
ABT     0.469198
ACN     0.525068
AIG     0.475104
dtype: float64

In [78]:
correlation_from_0 = abs(0 - correlation_pct_mean)
correlation_from_0 = sorted(correlation_from_0.items(), key=lambda x:x[1])

In [79]:
points_system = pd.DataFrame()
expected_lst = []
std_deviation_lst = []
correlation_lst = []

for i in expected_return_sorted:
    expected_lst.append(i[0])
    
for i in std_lst:
    std_deviation_lst.append(i[0])

for i in correlation_from_0:
    correlation_lst.append(i[0])

points_system['Expected Returns'] = expected_lst
points_system['Standard Deviation'] = std_deviation_lst
points_system['Correlation'] = correlation_lst
points_system.index += 1 

points_system = points_system.set_index([points_system.index[::-1]])

points_system.head()

,Expected Returns,Standard Deviation,Correlation
55,MON,MON,MON
54,SO,BMY,LMT
53,PM,MRK,OXY
52,AXP,KO,SLB
51,CVS,PEP,COP


In [80]:
score = pd.DataFrame()
    
score['Tickers'] = expected_lst

score['Points'] = None

for i in range(len(score)):
    score['Points'][i] = (int(points_system[points_system['Expected Returns'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Standard Deviation'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Correlation'] == expected_lst[i]].index.values)) 

score = score.sort_values("Points")
score = score[::-1]
score = score.reset_index(drop=True)
score.index += 1 

score.head()

,Tickers,Points
1,MON,165
2,SO,145
3,MO,142
4,PM,142
5,CL,135


In [81]:
number_of_stocks = 25

score_dropped = score.copy()
score_dropped.drop(labels = range(number_of_stocks + 1, len(score) + 1), inplace = True)
score_dropped.head()

,Tickers,Points
1,MON,165
2,SO,145
3,MO,142
4,PM,142
5,CL,135


In [82]:
min_weighting = 100/(2*number_of_stocks)
base_weighting_total = min_weighting * number_of_stocks

score_dropped['Base Weighting'] = min_weighting
weighting_sum = score_dropped['Points'].sum()

score_dropped['Points-based Weighting'] = (score_dropped['Points']/weighting_sum) * (100 - base_weighting_total)
score_dropped['Total Weighting'] = score_dropped['Base Weighting'] + score_dropped['Points-based Weighting']
score_dropped.head()

,Tickers,Points,Base Weighting,Points-based Weighting,Total Weighting
1,MON,165,2.0,2.850726,4.850726
2,SO,145,2.0,2.505183,4.505183
3,MO,142,2.0,2.453352,4.453352
4,PM,142,2.0,2.453352,4.453352
5,CL,135,2.0,2.332412,4.332412


In [83]:
tickers_list = score_dropped['Tickers']
Portfolio_Final = pd.DataFrame()

start_date = '2022-11-22'
end_date = '2022-11-23'

for x in tickers_list:
    stock_info = yf.Ticker(x)
    stock_hist =  stock_info.history(start = start_date, end = end_date)
    
    Portfolio_Final[x] = stock_hist['Close']
    #price_list.append(recent_history_close['2022-11-22']) 
    
Portfolio_Final = Portfolio_Final.transpose()
Portfolio_Final = Portfolio_Final.reset_index()
Portfolio_Final.index += 1

Portfolio_Final.head()

Date,index,2022-11-22 00:00:00
1,MON,10.010000
2,SO,65.949997
3,MO,45.160000
4,PM,97.160004
5,CL,76.370003


In [84]:
total_amount_to_spend = 500000
column_names = ['Ticker', 'Price']
Portfolio_Final.columns = column_names
#value_df = pd.DataFrame()
Portfolio_Final['Shares'] = (total_amount_to_spend*(score_dropped['Total Weighting']/100))/Portfolio_Final['Price']
Portfolio_Final['Value'] = Portfolio_Final['Shares'] * Portfolio_Final['Price']
Portfolio_Final['Weight'] = score_dropped['Total Weighting']
Portfolio_Final
#print(score_dropped['Total Weighting']/100)

,Ticker,Price,Shares,Value,Weight
1,MON,10.010000,2422.939824,24253.628196,4.850726
2,SO,65.949997,341.560527,22525.915688,4.505183
3,MO,45.160000,493.063749,22266.758811,4.453352
4,PM,97.160004,229.176183,22266.758811,4.453352
5,CL,76.370003,283.646179,21662.059433,4.332412
6,ABBV,159.789993,132.862709,21230.131306,4.246026
7,T,19.000000,1103.735496,20970.97443,4.194195
8,KO,62.349998,333.571831,20798.203179,4.159641
9,CVS,99.510002,205.533718,20452.660677,4.090532
10,PG,145.619995,140.452282,20452.660677,4.090532


In [85]:
Stocks_Final = pd.DataFrame()

Stocks_Final['Ticker'] = Portfolio_Final['Ticker'] 
Stocks_Final['Shares'] = Portfolio_Final['Shares']

Stocks_Final

,Ticker,Shares
1,MON,2422.939824
2,SO,341.560527
3,MO,493.063749
4,PM,229.176183
5,CL,283.646179
6,ABBV,132.862709
7,T,1103.735496
8,KO,333.571831
9,CVS,205.533718
10,PG,140.452282
